In [1]:
import numpy as np
import scipy
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

This notebook takes in noise data(mag field and angular velocity), analyzes its noise distribution, Allan Deivation, and calcuates the covariance matrix

In [2]:
df = pd.read_csv("IMU_noise_T_50ms.txt")
df=df[1000:100000]
df.index = np.arange(len(df))
w_x= df.iloc[:,0]
w_y= df.iloc[:,1]
w_z= df.iloc[:,2]

mag_x  =df.iloc[:,3]
mag_y  =df.iloc[:,4]
mag_z  =df.iloc[:,5]

dt = 0.001*50
t = np.arange(0,mag_x.size)*dt


cov_input = np.array([ mag_x,mag_y,mag_z,w_x,w_y,w_z,])

df.shape
df

cov_input.shape

cov_input

array([[ 2.926e+01,  2.906e+01,  2.888e+01, ...,  2.906e+01,  2.938e+01,
         2.909e+01],
       [ 6.378e+01,  6.367e+01,  6.343e+01, ...,  6.337e+01,  6.340e+01,
         6.349e+01],
       [-5.735e+01, -5.738e+01, -5.820e+01, ..., -5.779e+01, -5.753e+01,
        -5.709e+01],
       [ 3.000e-02,  3.000e-02,  4.000e-02, ...,  3.000e-02,  3.000e-02,
         3.000e-02],
       [ 1.000e-02,  1.000e-02,  1.000e-02, ...,  1.000e-02,  1.000e-02,
         1.000e-02],
       [-3.000e-02, -3.000e-02, -3.000e-02, ..., -3.000e-02, -3.000e-02,
        -3.000e-02]])

In [ ]:
#Allan Deviation
def AD(n, dt, data):
    tau = n * dt
    AV = 0
    SegAvg = []
    m = int(mag_x.size / n)

    for i in range(m):
        sum = 0
        for j in range(n):
            sum += data[i * n + j]
        SegAvg.append(sum / n)

    for i in range(len(SegAvg)):
        if i + 1 < len(SegAvg):
            AV += (SegAvg[i + 1] - SegAvg[i]) ** 2

    AV = AV * (1 / (2 * len(SegAvg) - 1))
    AD = AV ** (0.5)
    return AD

ADlists =[]

for j in range(6):
    nlist = []
    ADlist = []
    for i in tqdm(range(int(w_x.size/10))):
        if i>0:
            nlist.append(i)
            ADlist.append(AD(i,dt, df.iloc[:,j]))
    ADlists.append(ADlist)


 13%|█▎        | 1325/9900 [04:57<32:59,  4.33it/s]

In [ ]:
#Allan Deviation Function
taulist = np.array(nlist)*dt

def index2dim(i):
    if i==0:
        return "x"
    elif i==1:
        return "y"
    elif i==2:
        return "z"
    else:
        return "Undefined"

for i in range(3):
    plt.plot(taulist,ADlists[i], label=index2dim(i))
    plt.xscale("log", base=10)
    plt.yscale("log", base=10)
    plt.ylabel("Allan Deviation")
    plt.xlabel("tau log10 (time in sec)")
plt.title("Allan Deviation Function on Angular Velocity")
plt.legend()
plt.show()

for i in range(3):
    plt.plot(taulist,ADlists[i+3],label=index2dim(i))
    plt.xscale("log", base=10)
    plt.yscale("log", base=10)
    plt.ylabel("Allan Deviation")
    plt.xlabel("tau log10 (time in sec)")
plt.title("Allan Deviation Function on Magnetic Field")
plt.legend()
plt.show()


In [ ]:
f,pd_wx = scipy.signal.welch(w_x, 1/dt)
f,pd_wy = scipy.signal.welch(w_y, 1/dt)
f,pd_wz = scipy.signal.welch(w_z, 1/dt)

f,pd_mx = scipy.signal.welch(mag_x, 1/dt)
f,pd_my = scipy.signal.welch(mag_y, 1/dt)
f,pd_mz = scipy.signal.welch(mag_z, 1/dt)

plt.plot(f,pd_wx,label="w_x")
plt.plot(f,pd_wy,label="w_y")
plt.plot(f,pd_wz,label="w_z")

plt.ylabel("Magnitude (dB)")
plt.xlabel("Frequency (Hz)")
plt.title("PSD (Welch method) on Angular Velocity")
plt.legend()
plt.show()

plt.plot(f,pd_mx,label="mag_x")
plt.plot(f,pd_my,label="mag_y")
plt.plot(f,pd_mz,label="mag_z")

plt.ylabel("Magnitude (dB)")
plt.xlabel("Frequency (Hz)")
plt.title("PSD (Welch method) on Magnetic Field")
plt.legend()
plt.show()

In [ ]:

w_x_norm = w_x-np.mean(w_x)
w_y_norm = w_y-np.mean(w_y)
w_z_norm = w_z-np.mean(w_z)

n, bins, patches =plt.hist(w_x_norm, bins=20, histtype='bar')
plt.show()

n, bins, patches = plt.hist(w_y_norm, 20, histtype='bar')
plt.show()

n, bins, patches = plt.hist(w_z_norm, 20, histtype='bar')
plt.show()
# plt.plot(w_x,scipy.stats.norm.pdf(w_x),".")
# plt.show()
# plt.plot(w_y,scipy.stats.norm.pdf(w_y),".")
# plt.show()
# plt.plot(w_z,scipy.stats.norm.pdf(w_z),".")
# plt.show()

w_x_norm

In [ ]:
mag_x_norm = mag_x-np.mean(mag_x)
mag_y_norm = mag_y-np.mean(mag_y)
mag_z_norm = mag_z-np.mean(mag_z)

n, bins, patches =plt.hist(mag_x_norm, bins=100, histtype='bar')
plt.show()

n, bins, patches = plt.hist(mag_y_norm, 100, histtype='bar')
plt.show()

n, bins, patches = plt.hist(mag_z_norm, 100, histtype='bar')
plt.show()
# plt.plot(mag_x,scipy.stats.norm.pdf(mag_x),".")
# plt.show()
# plt.plot(mag_y,scipy.stats.norm.pdf(mag_y),".")
# plt.show()
# plt.plot(mag_z,scipy.stats.norm.pdf(mag_z),".")
# plt.show()

In [3]:
#Cov Matrix

def index2dim_total(i):
    if i==0:
        return mag_x
    if i==1:
        return mag_y
    if i==2:
        return mag_z
    if i==3:
        return w_x
    if i==4:
        return w_y
    if i==5:
        return w_z

def cov(a,b):
    a_mean = np.mean(a)
    b_mean = np.mean(b)

    return np.mean((a-a_mean)*(b-b_mean))

w_cov = np.zeros((3,3))
mag_cov = np.zeros((3,3))

total_cov = np.zeros((6,6))

for i in range(6):
    for j in range(6):
        total_cov[i][j] = cov(index2dim_total(i),index2dim_total(j))
print(total_cov)


[[ 2.02557174e-01  5.17509787e-03 -3.16666163e-02 -1.76501723e-04
  -3.74887387e-05 -7.75649668e-05]
 [ 5.17509787e-03  1.55387811e-01  1.07779379e-02 -2.90509018e-05
  -8.02923064e-06 -1.26276347e-05]
 [-3.16666163e-02  1.07779379e-02  3.93158713e-01  9.29620683e-05
   1.22495578e-05  5.67086399e-05]
 [-1.76501723e-04 -2.90509018e-05  9.29620683e-05  1.80159726e-05
  -2.27000306e-09 -6.07370830e-07]
 [-3.74887387e-05 -8.02923064e-06  1.22495578e-05 -2.27000306e-09
   6.70137291e-06  2.97295684e-08]
 [-7.75649668e-05 -1.26276347e-05  5.67086399e-05 -6.07370830e-07
   2.97295684e-08  8.52183425e-06]]


In [ ]:
print(np.cov(cov_input))